In [1]:
import requests 
from dotenv import load_dotenv
import os  
import pandas as pd 

In [2]:
# Load environment variables 
load_dotenv('env_var.env')

# Access environment variables 
subgraph_url = os.environ['SUBGRAPH_URL']
api_host = os.environ['API_HOST']
api_key = os.environ['API_KEY']

if 'SUBGRAPH_URL' in os.environ and 'API_HOST' in os.environ and 'API_KEY' in os.environ:
    print(True)
else:
    print(False)

True


### Get subgraph data and topology data to establish the peer_id-node_address-safe_address link 

In [3]:
def get_subgraph_data():
    """
    This function retrieves safe_address-node_address-balance links from the
    specified subgraph using pagination.
    """

    query = """
            query SafeNodeBalance($first: Int, $skip: Int) {
                safes(first: $first, skip: $skip) {
                    registeredNodesInNetworkRegistry {
                    node {
                        id
                    }
                    safe {
                        id
                        balance {
                        wxHoprBalance
                        }
                    }
                    }
                }
            }
        """

    data = {
        "query": query,
        "variables": {
            "first": 1000,
            "skip": 0,
        },
    }
    subgraph_dict = {}
    more_content_available = True
    pagination_skip_size = 1000

    while more_content_available:
        try:
            response = requests.post(subgraph_url, json=data)

            if response.status_code != 200:
                print(f"Received status code {response.status_code} when querying The Graph API")
                break

            json_data = response.json()

        except requests.exceptions.RequestException:
            print("An error occurred while sending the request to subgraph endpoint")
            return {}
        except ValueError:
            print("An error occurred while parsing the response as JSON from subgraph endpoint")
            return {}
        except Exception:
            print("An unexpected error occurred")
            return {}

        safes = json_data["data"]["safes"]
        for safe in safes:
            for node in safe["registeredNodesInNetworkRegistry"]:
                node_address = node["node"]["id"]
                wxHoprBalance = node["safe"]["balance"]["wxHoprBalance"]
                safe_address = node["safe"]["id"]
                subgraph_dict[node_address] = {
                    "safe_address": safe_address,
                    "wxHOPR_balance": wxHoprBalance,
                }

        # Increment skip for the next iteration
        data["variables"]["skip"] += pagination_skip_size
        more_content_available = len(safes) == pagination_skip_size

    return subgraph_dict

def get_unique_nodeAddress_peerId_aggbalance_links(api_host, api_key):
    """
    Returns a dict containing all unique source_peerId-source_address links.
    """
    channel_url = "http://{}:3001/api/v3/channels/?includingClosed=false&fullTopology=true".format(api_host)
    headers = {'X-Auth-Token': api_key}
    response = requests.request("GET", channel_url, headers=headers)

    if response.status_code != 200:
        print("Could not fetch channel information. Status code: {}".format(response.status_code))
        return {}
    
    response = response.json()

    if 'all' not in response:
            print("Response does not contain `all`")
            return {}

    peerid_address_aggbalance_links = {}
    for item in response["all"]:
        if "sourcePeerId" not in item or "sourceAddress" not in item:
            print("Response does not contain `source_peerid` or `source_address`")
            continue

        if "status" not in item:
            print("Response does not contain `status`")
            continue

        source_peer_id = item["sourcePeerId"]
        source_address = item["sourceAddress"]
        balance = int(item["balance"]) / 1e18

        if item["status"] != "Open":
            # Other Statuses: "Waiting for commitment", "Closed", "Pending to close"
            # Ensures that nodes must have at least 1 open channel in to receive ct
            continue

        if source_peer_id not in peerid_address_aggbalance_links:
            peerid_address_aggbalance_links[source_peer_id] = {
                "source_node_address": source_address,
                "channels_balance": balance,
            }

        else:
            peerid_address_aggbalance_links[source_peer_id][
                "channels_balance"
            ] += balance

    return peerid_address_aggbalance_links

In [4]:
subgraph_data = get_subgraph_data()
print(len(subgraph_data))

print_size = 10 

for key, value in list(subgraph_data.items())[:print_size]:
    print(f"{key}: {value}")


328
0xcdd7117148b6f2975815dd39d99bb9ed4553842b: {'safe_address': '0x00133125ccdf4ea1231a47e073c616f358b2d5a8', 'wxHOPR_balance': '25133.755553838447062483'}
0x2168fcd793a3967fa4bdd66f534c4fc811124439: {'safe_address': '0x01f1d2f347ea987b5cf3ed383146feda5265f38a', 'wxHOPR_balance': '46452.387878358780883268'}
0xed04f9fbf9160793fff7532df3860c70862bff4e: {'safe_address': '0x0420bd44fe87a855a11c9fd42b3f42203b03dec9', 'wxHOPR_balance': '29904.007400087394399044'}
0xfcc30ccecf890362d66194659f4850acbe84b08b: {'safe_address': '0x042ddd9d9b99ed1a08eb5c5a3feae5e7a1732e82', 'wxHOPR_balance': '20054'}
0xf3e7672a909fd8c113fc5c53dda1f38f79d7a184: {'safe_address': '0x04b21235a04d7468bdd79de8a68341b7be0a71fa', 'wxHOPR_balance': '61208.146712387571348229'}
0x06e7df53f76d5a0d3114e1ab6332a66b4e36cd86: {'safe_address': '0x04d516f717ac1e45af3cd9694c37be10470cfb28', 'wxHOPR_balance': '0'}
0xb7dde63eb9d91f0b6dc70f8d8ffe0f1187d3b3e1: {'safe_address': '0x05481f2df974592b30010ace07d2ab037e921f42', 'wxHOPR_balan

### Load message statistics 

In [5]:
message_statistics = pd.read_csv('message_statistics.csv', low_memory=False)

print(len(message_statistics))


196


In [6]:
topology_data = get_unique_nodeAddress_peerId_aggbalance_links(api_host, api_key)

print(len(topology_data))

print_size = 5 

for key, value in list(topology_data.items())[:print_size]:
    print(f"{key}: {value}")

221
12D3KooWAAZ3WnaWMuSQnQSFYnwx7g6W725DYmDY1QASZK1nni5B: {'source_node_address': '0xfa73af5609d354acd0f6e51f77c04d228c370813', 'channels_balance': 250.0}
12D3KooWKn3bLWMymSMznYt3YRrK8cwmYytEzwM1HoDgqkLcmeSu: {'source_node_address': '0x8de95cdc3291152381f24226422ed9c08beec146', 'channels_balance': 2176.0}
12D3KooWNJyWUuu9M9RNU8zTMyqCA8K1GqJwv8tjHsc2oYsJJajD: {'source_node_address': '0x9925b9846e6e289e1c7f8a0554bf6900757491ec', 'channels_balance': 140.0}
12D3KooWH9rfYNKMkNncYJxS7BH41ThPZUYe3FNkbfmJAa4n5r3x: {'source_node_address': '0x5a5bf3d3ce59cd304f198b86c1a78adfadf31f83', 'channels_balance': 10881.0}
12D3KooWC7Tub3aRmxUM7XjiozW5paXcySN9gWDDvJwNh98DTdUD: {'source_node_address': '0x07a52c8338edef4b6d3a180c1b75815469ff3a46', 'channels_balance': 1622.0}


In [8]:
def merge_topology_subgraph(topology_dict: dict, subgraph_dict: dict):
    """
    Merge metrics and subgraph data with the unique peer IDs, addresses,
    balance links.
    :param: topology_dict: A dict mapping peer IDs to node addresses.
    :param: subgraph_dict: A dict containing subgraph data with safe address as the key.
    :returns: A dict with peer ID as the key and the merged information.
    """
    merged_result = {}

    # Merge based on peer ID with the channel topology as the baseline
    for peer_id, data in topology_dict.items():
        seen_in_subgraph = False

        source_node_address = data["source_node_address"]
        if source_node_address in subgraph_dict:
            subgraph_data = subgraph_dict[source_node_address]
            data["safe_address"] = subgraph_data["safe_address"]
            data["safe_balance"] = float(subgraph_data["wxHOPR_balance"])
            data["total_balance"] = data["channels_balance"] + data["safe_balance"]

            seen_in_subgraph = True
            # print(f"Source node address for {peer_id} found in subgraph")

        if seen_in_subgraph:
            merged_result[peer_id] = data

    return merged_result

In [9]:
merged_data = merge_topology_subgraph(topology_data, subgraph_data)
print(len(merged_data))

print_size = 5 

for key, value in list(merged_data.items())[:print_size]:
    print(f"{key}: {value}")

221
12D3KooWAAZ3WnaWMuSQnQSFYnwx7g6W725DYmDY1QASZK1nni5B: {'source_node_address': '0xfa73af5609d354acd0f6e51f77c04d228c370813', 'channels_balance': 250.0, 'safe_address': '0x8832376a388cfcb58dab0cce249f65b86041e4bc', 'safe_balance': 9798.21884352485, 'total_balance': 10048.21884352485}
12D3KooWKn3bLWMymSMznYt3YRrK8cwmYytEzwM1HoDgqkLcmeSu: {'source_node_address': '0x8de95cdc3291152381f24226422ed9c08beec146', 'channels_balance': 2176.0, 'safe_address': '0x4d219619fa660c02236c89e5de46843bdeb41233', 'safe_balance': 86880.0, 'total_balance': 89056.0}
12D3KooWNJyWUuu9M9RNU8zTMyqCA8K1GqJwv8tjHsc2oYsJJajD: {'source_node_address': '0x9925b9846e6e289e1c7f8a0554bf6900757491ec', 'channels_balance': 140.0, 'safe_address': '0x5ce3ca7ae22be88a960e0847845d9e13e8b5f23d', 'safe_balance': 74975.0, 'total_balance': 75115.0}
12D3KooWH9rfYNKMkNncYJxS7BH41ThPZUYe3FNkbfmJAa4n5r3x: {'source_node_address': '0x5a5bf3d3ce59cd304f198b86c1a78adfadf31f83', 'channels_balance': 10881.0, 'safe_address': '0xdf9be8bdb5ae

### Merge message statistics and the merged subgraph topology data

In [15]:
# Create a DataFrame from the dictionary
df_merged_data = pd.DataFrame.from_dict(merged_data, orient='index')

# Reset the index
df_merged_data.reset_index(inplace=True)

# Rename the columns
df_merged_data.columns = ['peer_id', 'source_node_address', 'channels_balance', 'safe_address', 'safe_balance', 'total_balance']

# Print the DataFrame
display(df_merged_data)

,peer_id,source_node_address,channels_balance,safe_address,safe_balance,total_balance
0,12D3KooWAAZ3WnaWMuSQnQSFYnwx7g6W725DYmDY1QASZK...,0xfa73af5609d354acd0f6e51f77c04d228c370813,250.0,0x8832376a388cfcb58dab0cce249f65b86041e4bc,9798.218844,10048.218844
1,12D3KooWKn3bLWMymSMznYt3YRrK8cwmYytEzwM1HoDgqk...,0x8de95cdc3291152381f24226422ed9c08beec146,2176.0,0x4d219619fa660c02236c89e5de46843bdeb41233,86880.000000,89056.000000
2,12D3KooWNJyWUuu9M9RNU8zTMyqCA8K1GqJwv8tjHsc2oY...,0x9925b9846e6e289e1c7f8a0554bf6900757491ec,140.0,0x5ce3ca7ae22be88a960e0847845d9e13e8b5f23d,74975.000000,75115.000000
3,12D3KooWH9rfYNKMkNncYJxS7BH41ThPZUYe3FNkbfmJAa...,0x5a5bf3d3ce59cd304f198b86c1a78adfadf31f83,10881.0,0xdf9be8bdb5ae4a130e861e5158c95667e7b2c0cb,11026.000000,21907.000000
4,12D3KooWC7Tub3aRmxUM7XjiozW5paXcySN9gWDDvJwNh9...,0x07a52c8338edef4b6d3a180c1b75815469ff3a46,1622.0,0x84600a20ea20d4cc139e0483650320d8d26c03d1,14897.000000,16519.000000
...,...,...,...,...,...,...
216,12D3KooWPY4SJ7Y54EDbZhLeaRvCzo4QLgKwY9hwcmLKuJ...,0xe9e76aa349e3ca33d37cd9713a64608cd029aa97,20.0,0xc4ad115c1f71b79563d27bfdc4208e56f0c0aec3,10032.000000,10052.000000
217,12D3KooWNrVRJbP7aqacxEt6DZjzoz7SRzawRvY2gjZTnH...,0x4663c9beee1c95205dc79a0d7c019753bc9e4759,10.0,0x268195238449e5521b56a7f34b951e01416c152d,10033.000000,10043.000000
218,12D3KooWNeRYMmbBkiv7fvNS9Eh1NFoEQHJTtabEZGf7Mt...,0x0b719bc3ce484321b7ded5fe5a6523ca400a815b,1.0,0x3510d09b5b718220bf7df02fe374373744aa611a,10032.000000,10033.000000
219,12D3KooWHyJe3bNyT64ymbCUuqniitmbhKUEZfMum8eZhy...,0x4ca9d39fc1242b806c765c742d601eb32f691f5b,1.0,0xd73716d02202ab88418a7040861365ce35208293,10028.000000,10029.000000


In [18]:
elidgible_peers_for_airdrop = message_statistics.merge(df_merged_data, how='left', left_on='peer_id'
                                                    , right_on='peer_id')


print(elidgible_peers_for_airdrop.columns)
print(len(elidgible_peers_for_airdrop))



Index(['peer_id', 'expected_messages', 'issued_messages',
       'source_node_address', 'channels_balance', 'safe_address',
       'safe_balance', 'total_balance'],
      dtype='object')
196


In [19]:
elidgible_peers_for_airdrop_01 = elidgible_peers_for_airdrop[~elidgible_peers_for_airdrop['source_node_address'].isnull()] 
print(len(elidgible_peers_for_airdrop_01))

194


In [20]:
elidgible_peers_for_airdrop_01.to_excel('elidgible_peers_for_airdrop.xlsx', index=False)